<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="3em" style="font-weight:bold;color:#3f8dbf;">탐색적 데이터 분석: 1. 데이터의 출처와 주제에 대해 이해</font><br>

### 전체 판매 프로세스
1. 해당 쇼핑몰에 중소업체가 계약을 맺고
2. 중소업체가 해당 쇼핑몰에 직접 상품을 올리고
2. 고객이 구매하면, 중소업체가 Olist가 제공하는 물류 파트너를 활용해서 배송을 하고,
3. 고객이 상품을 받으면, 고객에게 이메일 survey 가 전송되고,
4. 고객이 이메일 survey 에 별점과 커멘트를 남겨서 제출하게 됨
    
### 데이터 출처
- 브라질에서 가장 큰 백화점의 이커머스 쇼핑몰 (https://olist.com/)
  - 2016년도부터 2018년도 9월까지의 100k 개의 구매 데이터 정보
  - 구매 상태, 가격, 지불수단, 물류 관련, 리뷰관련, 상품 정보, 구매자 지역 관련 정보

</div>

### [ 1. 얼마나 많은 고객이 있는가? ]

In [10]:
import pandas as pd
path = './olist_ecommerce_data/'
products = pd.read_csv( path + 'olist_products_dataset.csv', encoding = 'utf-8-sig')
customers = pd.read_csv( path + 'olist_customers_dataset.csv', encoding = 'utf-8-sig')
geolocation = pd.read_csv( path + 'olist_geolocation_dataset.csv', encoding = 'utf-8-sig')
order_items = pd.read_csv( path + 'olist_order_items_dataset.csv', encoding = 'utf-8-sig')
payments = pd.read_csv( path + 'olist_order_payments_dataset.csv', encoding = 'utf-8-sig')
reviews = pd.read_csv( path + 'olist_order_reviews_dataset.csv', encoding = 'utf-8-sig')
orders = pd.read_csv( path + 'olist_orders_dataset.csv', encoding = 'utf-8-sig')
sellers = pd.read_csv( path + 'olist_sellers_dataset.csv', encoding = 'utf-8-sig')
category_name = pd.read_csv( path + 'product_category_name_translation.csv', encoding = 'utf-8-sig')

In [11]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [16]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
위의 데이터프레임만 관측하였을때, customer_id 와 customer_unique_id 중 고객의 지표가 될 수 있는 것이 무엇인지 정확히 파악이 안된다. 따라서 둘중하나는 다른 하나에 속하는 중복 id라 예측할 수 있다.

따라서, 각 아이디의 중복값이 최대 1인 객체가 고객 한명이라고 판단할 수 있다.

- value_counts() : 각각의 값이 전체 series에서 각각 몇개가 있는지 알려준다.

In [14]:
customers['customer_id'].value_counts().max()

1

In [15]:
customers['customer_unique_id'].value_counts().max()

17

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
    위의 결과를 미루어 봤을 때, customer_id 하나가 고객 한명이라고 판단할 수 있다.

- nunique() : unique 한 값의 갯수를 알려준다. ex) A,B,C,C,D -> unique값 4개

In [19]:
# 고객의 총 수는 99441로 추측할 수 있다.
customers['customer_id'].nunique()

99441

In [18]:
# 실제로 customer_unique_id의 고유값이 중복으로 인해 customer_id 보다 적음을 확인 할 수 있다,
customers['customer_unique_id'].nunique()

96096

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="3em" style="font-weight:bold;color:#3f8dbf;">고객 분석1: 실제 고객 수는 99441 로 볼 수 있음</font><br>
</div>

### [ 2. 고객은 어디에 주로 사는가? ]

방법 1

In [28]:
customers_location=customers[['customer_id', 'customer_city']].groupby('customer_city').count().sort_values(by = 'customer_id',ascending=False)
customers_location.head(20)

,customer_id
customer_city,
sao paulo,15540
rio de janeiro,6882
belo horizonte,2773
brasilia,2131
curitiba,1521
campinas,1444
porto alegre,1379
salvador,1245
guarulhos,1189


방법 2

In [31]:
customers_location = customers.groupby('customer_city')['customer_id'].nunique().sort_values(ascending = False)
customers_location.head(20)

customer_city
sao paulo                15540
rio de janeiro            6882
belo horizonte            2773
brasilia                  2131
curitiba                  1521
campinas                  1444
porto alegre              1379
salvador                  1245
guarulhos                 1189
sao bernardo do campo      938
niteroi                    849
santo andre                797
osasco                     746
santos                     713
goiania                    692
sao jose dos campos        691
fortaleza                  654
sorocaba                   633
recife                     613
florianopolis              570
Name: customer_id, dtype: int64

시각화

In [33]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected = True)

customers_location_top20 = customers_location.head(20)

# 절대빈도로 표현하기 위해 bar그래프를 차용한다.
customers_location_top20.iplot( kind = 'bar', theme = 'white')

In [35]:
customers_location_top20

customer_city
sao paulo                15540
rio de janeiro            6882
belo horizonte            2773
brasilia                  2131
curitiba                  1521
campinas                  1444
porto alegre              1379
salvador                  1245
guarulhos                 1189
sao bernardo do campo      938
niteroi                    849
santo andre                797
osasco                     746
santos                     713
goiania                    692
sao jose dos campos        691
fortaleza                  654
sorocaba                   633
recife                     613
florianopolis              570
Name: customer_id, dtype: int64

In [36]:
for index, location in enumerate(list(customers_location_top20.index)):
    print('TOP', index+1, ':', location)

TOP 1 : sao paulo
TOP 2 : rio de janeiro
TOP 3 : belo horizonte
TOP 4 : brasilia
TOP 5 : curitiba
TOP 6 : campinas
TOP 7 : porto alegre
TOP 8 : salvador
TOP 9 : guarulhos
TOP 10 : sao bernardo do campo
TOP 11 : niteroi
TOP 12 : santo andre
TOP 13 : osasco
TOP 14 : santos
TOP 15 : goiania
TOP 16 : sao jose dos campos
TOP 17 : fortaleza
TOP 18 : sorocaba
TOP 19 : recife
TOP 20 : florianopolis


<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="3em" style="font-weight:bold;color:#3f8dbf;">고객 분석2: 주 고객층이 사는 지역은 sao paulo 임을 확인 할 수 있다</font><br>
</div>

### [ 3. 고객들의 주 지불방법 ] 

In [38]:
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [43]:
# 없는 데이터 없는지 확인하는거 습관화하기
# 상기 : isnull() 메소드는 결측값의 경우 True로 나타남
# 결측값이 없는 것을 확인 가능
payments.isnull().sum()

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

- 중복된 지불 방법들을 unique() 함수로 유일값만 확인하게 할 수 있다.

In [46]:
payments['payment_type'].unique()

array(['credit_card', 'boleto', 'voucher', 'debit_card', 'not_defined'],
      dtype=object)

- 위에서 지불 방법 목록들을 보면 not_defined가 확인되는데, 이를 제외한 값을 가진 행 삭제하여 데이터를 관측

In [47]:
payments = payments[payments['payment_type'] != 'not_defined' ]

In [58]:
payment_type_count = payments.groupby('payment_type')['order_id'].nunique().sort_values(ascending = False)
payment_type_count.head()

payment_type
credit_card    76505
boleto         19784
voucher         3866
debit_card      1528
Name: order_id, dtype: int64

In [59]:
payment_type_count.iplot(kind = 'bar', theme = 'white')

In [61]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Pie(
    labels = payment_type_count.index, values = payment_type_count.values
    )
)

fig.update_layout(
    {
        'title' : {
            'text' : 'Payment Type Analysis',
            'font' : {
                'size' : 15
            }
        },
        'showlegend' : True
    }
)

fig.show()

조금더 상세하게

In [64]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Pie(
    labels = payment_type_count.index, 
    values = payment_type_count.values,
    # 파이그래프에 label과 percent를 동시에 나타나게 함
    textinfo = 'label+percent',
    # 파이그래프에서 라벨을 어떻게 배치하는지에 관해서
    insidetextorientation = 'radial'
    )
)

fig.update_layout(
    {
        'title' : {
            'text' : 'Payment Type Analysis',
            'x' : 0.5,
            'y' : 0.9,
            'font' : {
                'size' : 15
            }
        },
        'showlegend' : True
    }
)

fig.show()